In [1]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

In [2]:
training_file = 'TrainingData.pkl'
testing_file= 'TestData.pkl'
# testing_file = ?

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
# with open(validation_file, mode='rb') as f:
#     valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = torch.tensor(train['features']), torch.tensor(train['labels'])
# X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = torch.tensor(test['features']), torch.tensor(test['labels'])

In [3]:
transform = transforms.Compose([transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

dataset = datasets.ImageFolder(root='Training', transform=transform)
total = len(dataset)
train_c = 0.85 * len(dataset)
valid_c = 0.15 * len(dataset)

train_data, valid_data = random_split(dataset,[int(train_c),int(valid_c)])
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128,shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=1, shuffle=True)

## Build Model

In [4]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.cnn1 = nn.Conv2d(in_channels= 3, out_channels= 6, kernel_size= 2, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride = 1,padding=0)

        self.cnn2 = nn.Conv2d(in_channels= 6, out_channels= 8, kernel_size= 2, padding=2, stride=1 )
        self.maxpool2 = nn.MaxPool2d(kernel_size=4, stride=1)

        self.fc1 = nn.Linear(16384//2,43)
    

    def forward(self,x):
        x = self.cnn1(x)
        x = torch.relu(x)
        x = self.maxpool1(x)

        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x
    
    def activations(self,x):
        z1 = self.cnn1(x)
        a1 = torch.relu(x)
        out = self.maxpoo1(a1)

        z2 = self.cnn2(out)
        a2 = torch.relu(z2)
        out = out.view(out.size(0),-1)
        return z1,a1, z2, a2, out


In [5]:
model = CNN()
model
if torch.cuda.is_available():
    print(True)
    model = model.cuda()
else:
    print("Cuda is not avaliable")

True


In [6]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [8]:
for epoch in range(10):  # 遍历数据集多次
    
    running_loss = 0.0
    total_samples = 0
    running_corrects= 0
    Valid_accuracy = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            z = model(inputs)
            _, preds = torch.max(z, 1)
            Valid_accuracy += (preds == labels).sum().item()
        Valid_accuracy = Valid_accuracy/(len(valid_data))
        print(f'Epoch {epoch+1}/{10} -Valid Accuracy: {Valid_accuracy:.4f}')
    for inputs, labels in trainloader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()  

        outputs = model(inputs)  
        loss = criterion(outputs, labels) 
        loss.backward()  
        optimizer.step() 
        _, preds = torch.max(outputs, 1)
        total_samples += inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)


        running_loss += loss.item()
    epoch_acc = running_corrects.double() / len(train_data)
    print(f'Epoch {epoch+1}/{10} - Loss: {running_loss:.4f}, Accuracy: {epoch_acc:.4f}')
    Valid_accuracy = 0






Epoch 1/10 -Valid Accuracy: 0.0073
Epoch 1/10 - Loss: 378.9187, Accuracy: 0.4284
Epoch 2/10 -Valid Accuracy: 0.7062
Epoch 2/10 - Loss: 128.6569, Accuracy: 0.7984
Epoch 3/10 -Valid Accuracy: 0.8654
Epoch 3/10 - Loss: 73.8361, Accuracy: 0.8910
Epoch 4/10 -Valid Accuracy: 0.9117
Epoch 4/10 - Loss: 52.4091, Accuracy: 0.9204
Epoch 5/10 -Valid Accuracy: 0.9219
Epoch 5/10 - Loss: 40.2290, Accuracy: 0.9390
Epoch 6/10 -Valid Accuracy: 0.9387
Epoch 6/10 - Loss: 32.3752, Accuracy: 0.9502
Epoch 7/10 -Valid Accuracy: 0.9552
Epoch 7/10 - Loss: 28.1550, Accuracy: 0.9565
Epoch 8/10 -Valid Accuracy: 0.9552
Epoch 8/10 - Loss: 24.0451, Accuracy: 0.9645
Epoch 9/10 -Valid Accuracy: 0.9429
Epoch 9/10 - Loss: 20.0505, Accuracy: 0.9685
Epoch 10/10 -Valid Accuracy: 0.9602
Epoch 10/10 - Loss: 16.4095, Accuracy: 0.9763
